Parallel Batch Evalulation
=======

We want to parallize the calculation in the batch mode.

In [ ]:
from __future__ import print_function

import sftrident as sf
from sftrident.amplitude import amplitude
import sftrident.coordinate as coord
import numpy as np
import time
timesSingle = {}
timesParallel = {}



In [ ]:
mass = 1.0
#Narr1 = (2,3,20,5,2,7,8)
Narr1 = (2,3,4,5,6,7,8)
frame = 'lab'
tempDPHI = 500.0
points= np.prod(Narr1)
print("points:",points)

In [ ]:
def buildArr_lcc_cart(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return sf.util.cartesian_product(ss,p1m,p1x,p1y,p2m,p2x,p2y)



configBatch = {'a0':1e-4,'coordSys':'lcc','mass':1.0,'xi':0.0,'dPhi':tempDPHI,'frame':frame,'mode':'batch'}


start = time.time()
cartLCC = buildArr_lcc_cart(*Narr1)
#arrLCC = buildArr_lcc_single(*Narr1)
timesSingle['initArr'] = time.time() - start

ampObjBatch = amplitude(**configBatch)

start = time.time()
resBatch = ampObjBatch(cartLCC)
timesSingle['calc']=time.time() - start
print(resBatch.shape)

In [ ]:
import pathos.multiprocessing as multiprocessing

class parallelintegrand(object):
    """ Convert (batch) integrand into multiprocessor integrand.

    Integrand should return a numpy array.
    """
    def __init__(self, fcn, nproc=4):
        " Save integrand; create pool of nproc processes. "
        self.fcn = fcn
        self.nproc = nproc
        self.pool = multiprocessing.Pool(processes=nproc)
    def __del__(self):
        " Standard cleanup. "
        self.pool.close()
        self.pool.join()
    def __call__(self, x):
        " Divide x into self.nproc chunks, feeding one to each process. "
        nx = x.shape[0] // self.nproc + 1
        # launch evaluation of self.fcn for each chunk, in parallel
        results = self.pool.map(
            self.fcn,
            [x[i*nx : (i+1)*nx] for i in range(self.nproc)],
            1,
            )
        # convert list of results into a single numpy array
        return np.concatenate(results)


start = time.time() 
ampObjParallel = parallelintegrand(ampObjBatch,4)
timesParallel['initArr']=time.time() - start

start = time.time()
resParallel = ampObjParallel(cartLCC)
timesParallel['calc']=time.time() - start


In [ ]:
print(np.abs(resParallel - resBatch).max())
physPoints = len(resBatch[resBatch>0.0])
print(physPoints)

In [ ]:
for el in timesSingle.iterkeys():
    print(el,':')
    print("\tbatch:    ","%1.2e"%timesSingle[el],"(%1.2e)"%(timesSingle[el]/(float(physPoints))))
    print("\tparallel: ","%1.2e"%timesParallel[el],"(%1.2e)"%(timesParallel[el]/(float(physPoints))))
    print()

In [ ]:
import vegas


@vegas.batchintegrand
def vegasIntegrand(kinPara):
    if len(kinPara.shape)==1:
        kinPara = np.array([kinPara])
    #print(kinPara.shape)
    ss=np.array([3.353]*len(kinPara))
    kinArr = np.column_stack([ss,kinPara])
    #print(kinArr.shape)
    res = ampObjBatch(kinArr)
    #print(res)
    return res

#integ = vegas.Integrator(2 * [(0,1)])
p2mLim = (0.1,1.5)
p2xLim = (-3.5,3.5)
p2yLim = (-3.5,3.5)
p3mLim = (0.1,1.5)
p3xLim = (-3.5,3.5)
p3yLim = (-3.5,3.5)

start = time.time()
integ = vegas.Integrator([p2mLim, p2xLim, p2yLim, p3mLim, p3xLim, p3yLim])
timesSingle['integInit'] = time.time() - start

start = time.time()
training = integ(vegasIntegrand, nitn=10, neval=2000)
timesSingle['integTraining'] = time.time() - start

start = time.time()
result = integ(vegasIntegrand, nitn=10, neval=10000)
timesSingle['integ'] = time.time() - start

print(result.summary())


In [ ]:
A = np.array([[0, 1, 2], [0, 2, 0]])
print(A)
newrow = [10,20]
a = np.column_stack((newrow,A))
print(a)


In [ ]:
a = np.linspace(1,2,10)
a=np.array([a])
print(a.shape)
print(a[:,0])

In [ ]:

import vegas
import numpy as np

@vegas.batchintegrand
def f_batch(x):
    #print(x.shape)
    # evaluate integrand at multiple points simultaneously
    dim = x.shape[1]
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[:, d] - 0.5) ** 2
    res = np.exp(-100. * dx2) * norm
    #print(res.shape)
    return res

integ = vegas.Integrator(4 * [[0, 1]])

integ(f_batch, nitn=10, neval=2e5)
result = integ(f_batch, nitn=10, neval=2e5)
print(result.summary())
print('result = %s   Q = %.2f' % (result, result.Q))

In [ ]:
for el in timesSingle.iterkeys():
    print(el,':')
    print("\tbatch:    ","%1.2e"%timesSingle[el],"(%1.2e)"%(timesSingle[el]/(float(physPoints))))
    print()

In [ ]:
import vegas


@vegas.batchintegrand
def vegasIntegrand(kinPara):
    if len(kinPara.shape)==1:
        kinPara = np.array([kinPara])
    #print(kinPara.shape)
    ss=np.array([4.1]*len(kinPara))
    kinArr = np.column_stack([ss,kinPara])
    #print(kinArr.shape)
    res = ampObjBatch(kinArr)
    #print(res)
    return res

#integ = vegas.Integrator(2 * [(0,1)])
p2mLim = (0.1,1.5)
p2xLim = (-3.5,3.5)
p2yLim = (-3.5,3.5)
p3mLim = (0.1,1.5)
p3xLim = (-3.5,3.5)
p3yLim = (-3.5,3.5)

start = time.time()
integ = vegas.Integrator([p2mLim, p2xLim, p2yLim, p3mLim, p3xLim, p3yLim])
timesSingle['integInit'] = time.time() - start

start = time.time()
training = integ(vegasIntegrand, nitn=10, neval=5000)
timesSingle['integTraining'] = time.time() - start

start = time.time()
result = integ(vegasIntegrand, nitn=10, neval=20000)
timesSingle['integ'] = time.time() - start

print(result.summary())

In [ ]:
for el in timesSingle.iterkeys():
    print(el,':')
    print("\tbatch:    ","%1.2e"%timesSingle[el],"(%1.2e)"%(timesSingle[el]/(float(physPoints))))
    print()

In [ ]:
import vegas


@vegas.batchintegrand
def vegasIntegrand(kinPara):
    if len(kinPara.shape)==1:
        kinPara = np.array([kinPara])
    #print(kinPara.shape)
    ss=np.array([3.353]*len(kinPara))
    kinArr = np.column_stack([ss,kinPara])
    #print(kinArr.shape)
    res = ampObjBatch(kinArr)
    #print(res)
    return res

#integ = vegas.Integrator(2 * [(0,1)])
p2mLim = (0.0001,1.5)
p2xLim = (-6.0,6.0)
p2yLim = (-6.0,6.0)
p3mLim = (0.0001,1.5)
p3xLim = (-6.0,6.0)
p3yLim = (-6.0,6.0)

start = time.time()
integ = vegas.Integrator([p2mLim, p2xLim, p2yLim, p3mLim, p3xLim, p3yLim])
timesSingle['integInit'] = time.time() - start

start = time.time()
training = integ(vegasIntegrand, nitn=10, neval=5000)
timesSingle['integTraining'] = time.time() - start

start = time.time()
result = integ(vegasIntegrand, nitn=10, neval=20000)
timesSingle['integ'] = time.time() - start

print(result.summary())

In [ ]:
for el in timesSingle.iterkeys():
    print(el,':')
    print("\tbatch:    ","%1.2e"%timesSingle[el],"(%1.2e)"%(timesSingle[el]/(float(physPoints))))
    print()

In [ ]:
print(training.summary())